# Introduction to ExoCTK
Authors: Jules Fowler, Joe Filippazzo, Matthew Bourque, Kevin Stevenson

The Exoplanet Characterization Tool Kit (ExoCTK) is an open source Python 3.5+ package that serves as a one-stop-shop for observation planning, data reduction, and scientific analysis of transiting exoplanet systems.

In this notebook, we will cover the installation of the software as well as an end-to-end example that demonstrates the core functionalities of each tool provided by the ExoCTK package.

# Installation
You must first have a working installation of ``anaconda`` or ``miniconda`` for Python 3.  If you do not yet have this on your system, you can visit the following links for download and installation instructions:

- [Anaconda](https://www.anaconda.com/download/)
- [Miniconda](https://conda.io/en/latest/miniconda.html)

To obtain the latest `exoctk` package with the necessary environment files, you should clone the repository directly from GitHub:

```
git clone https://github.com/ExoCTK/exoctk.git
cd exoctk
```

To obtain the `exoctk` data, visit the shared [ExoCTK DropBox](https://www.dropbox.com/sh/p9c49urdbkmn5rz/AACFIjeAphOPqcCLtqd1v_W9a?dl=0), download the folders, and unzip them into a directory called `exoctk_data/`.  You may need to download  `modelgrid` seperately because it's the largest of the bunch and some computers have trouble with large .zip files.  Next, export an environment variable for `EXOCTK_DATA`. For Mac OS/Linux, add the line

```
export EXOCTK_DATA='/path/to/your/unzipped/directory/exoctk_data/'
```

to your `.bashrc` or `.bash_profile`. Don't forget to execute your `.bashrc` or `.bash_profile` to implement the changes in your terminal.

```
source ~/.bash_profile
```

For Windows, add an evironment variable using System Utility. 


You can install the ExoCTK `conda` environment via the `env/environment-<PYTHON_VERSION>.yml` files (relative to the parent directory of where the `exoctk` repository was installed).  Note that there are separate environment files for each version of `python` that `exoctk` supports.  For this demo, we will be using version 3.6. Ensure that `conda` is up to date, create the `exoctk` environment with Python version 3.6, and activate it with:

```
conda update conda
conda activate base
conda env create -f env/environment-3.6.yml
conda activate exoctk-3.6
conda install ipykernel
python -m ipykernel install --user --name exoctk-3.6 --display-name "exoctk-3.6"
```

Finally, install `exoctk` by running the `exoctk` setup script:

```
python setup.py install
```
# Kernl
Before getting started, make sure that you are using the `exoctk-3.6` kernel be selecting it via `Kernel` -> `Change kernel`.

# Imports
Here are a few basic imports we'll need later on...

In [1]:
# NOTE -- we're going to need to include install instructions for hotsoss as well.
import os
import numpy as np
from bokeh.io import output_notebook
from bokeh.plotting import show
#from hotsoss import plotting as plt
output_notebook()

Loading BokehJS ...

# Ask a compelling science question
Successful proposals require a well-posed science question with testable hypothesis.  For this end-to-end demonstration, let's ask a simple question: "Where's the methane?"  More specifically, we want to know if planets with Teq < 1000 have methane in their atmospheres, as predicted by chemical equilibrium models.

# Find the best target that answers your question
Next, let's use the [Exo.MAST Observability Table](https://catalogs.mast.stsci.edu/eaot) to find the highest signal-to-noise target in transmission that is likely to have methane.  Applying the filter: `Teq < 1000 K` and sorting on `K-Band Transmission SNR` shows that WASP-107b is our best target.

# Transiting exoplanet system example
Let's grab all of WASP-107's relevant system parameters using the [ExoMAST](https://exo.mast.stsci.edu) service lookup in ExoCTK.  There are a lot of parameters, so let's just print the planet name and temperature.

In [2]:
# Get the orbital parameters
from exoctk.utils import get_target_data
wasp107b_params, url = get_target_data('WASP-107b')
#print(wasp107b_params.keys())
print(wasp107b_params['planet_name'], wasp107b_params['Tp'])

WASP-107 b 739.99


In [3]:
# Let's simulate a lightcurve of WASP-107b to see what it might look like
#from exoctk.lightcurve_fitting.simulations import simulate_lightcurve
#npts = 1000
#wasp107b_time, wasp107b_flux, wasp107b_par = simulate_lightcurve('WASP-107b', snr=4000., npts=npts, plot=True)

# Observation Planning with the Groups and Integrations Calculator
This section will describe planning an observation, including calculating the correct number of groups and Integrations to enter into the Astronomers Proposal Tool (APT).

## Groups and Integrations

To plan an observation with APT, an observer needs to calculate the groups and integerations required for their observation. Each JWST observation consists of some amount of overhead (time to reset the detector, etc), and a discrete number of integrations, each made up of 'n' frames. For time-series observations, there is always only one frame per group, so each integration also has 'n' groups.  The total observation time is determined by the frame time of your given instrument and mode, the number of groups per integration, and the number of integrations in your exposure.  An observation with too many groups per integration will saturate the detector, but too few groups per integration will lead to inefficient observations (i.e. low duty cycles). 

This part of the demo will teach you how to calculate the groups and integrations of our target, WASP 107b, and send it through APT. 

In [4]:
# Imports for Groups and Integrations
import astropy.units as u
import json

from exoctk.groups_integrations.groups_integrations import perform_calculation

# Specify path to data file
EXOCTK_DATA = os.environ.get('EXOCTK_DATA')
if EXOCTK_DATA == '':
    GROUPS_INTEGRATIONS_DIR = '/path/to/your/local/copy/integrations_groups_data.json'
else:
    GROUPS_INTEGRATIONS_DIR = GROUPS_INTEGRATIONS_DIR = os.path.join(EXOCTK_DATA, 'groups_integrations/groups_integrations.json')

In [5]:
# Open the file
with open(GROUPS_INTEGRATIONS_DIR) as f:
    parameter_space = json.load(f)
    
# Print the target acquisition (TA) and science observation modes
modes = [('science', 'sci_sat'), ('TA', 'ta_sat')]
for mode in modes:
    print('\n')
    print('Available modes for {} :'.format(mode[0]))
    print('------------------------')
    inss = list(parameter_space[mode[1]].keys())
    print('Instruments : {}'.format(inss))
    for ins in inss:
        filts = list(parameter_space[mode[1]][ins].keys())
        print('Filters for {} : {}'.format(ins, filts))
        subs = list(parameter_space[mode[1]][ins][filts[0]].keys())
        print('Subarrays for {} : {}'.format(ins, subs))

# Print the available Phoenix models
print('\n')
print('Phoenix model keys :')
print('---------------------')
print(list(parameter_space['ta_sat'][inss[-1]][filts[0]][subs[0]].keys()))

print('\n')
print('Magnitude sampling :')
print('--------------------')
print(parameter_space['mags'])



Available modes for science :
------------------------
Instruments : ['niriss', 'nircam', 'nirspec', 'miri']
Filters for niriss : ['soss']
Subarrays for niriss : ['substrip96', 'substrip256']
Filters for nircam : ['f444w', 'f277w', 'f322w2']
Subarrays for nircam : ['subgrism128', 'full', 'subgrism256', 'subgrism64']
Filters for nirspec : ['f070lp_g140h', 'f290lp_g395h', 'f170lp_g235m', 'f170lp_g235h', 'f070lp_g140m', 'f100lp_g140h', 'f100lp_g140m', 'f290lp_g395m']
Subarrays for nirspec : ['sub1024a', 'sub1024b', 'sub2048', 'sub512']
Filters for miri : ['lrs']
Subarrays for miri : ['slitlessprism']


Available modes for TA :
------------------------
Instruments : ['niriss', 'nircam', 'nirspec', 'miri']
Filters for niriss : ['f480m']
Subarrays for niriss : ['im', 'nrm']
Filters for nircam : ['f335m']
Subarrays for nircam : ['sub32tats']
Filters for nirspec : ['f110w', 'f140x', 'clear']
Subarrays for nirspec : ['sub2048', 'full', 'sub32']
Filters for miri : ['f1500w', 'f100w', 'f560w']


In [6]:
# Initialize a dictionary of inputs
# When working with the webform, this is created for you
# This information for WASP 107b is available at : https://exo.mast.stsci.edu/exomast_planet.html?planet=WASP19b
parameters = {}

# Source parameters
parameters['mag'] = wasp107b_params['Kmag'] # Stellar mangnitude
parameters['band'] = 'k' # only K band vega mag for now
parameters['mod'] = 'k5v' # Phoenix model per last section

# Observation specifics 
obs_time = wasp107b_params['transit_duration']*u.d.to('h')*3 + 1 # 3*transit duration, converted to hours, plus an extra hour of time
parameters['obs_time'] = obs_time
parameters['n_group'] = 'optimize' # 'optimize', or positive integer

# Detector setup -- within the modes of the last section
parameters['ins'] = 'niriss'
# For science observation
parameters['filt'] = 'soss'
parameters['subarray'] = 'substrip256'
# And target acquisition
parameters['filt_ta'] = 'f480m'
parameters['subarray_ta'] = 'im'

# Saturation level
parameters['sat_mode'] = 'well' # 'well', for full well fraction, or 'counts' 
parameters['sat_max'] = .80 # < 1 for fullwell, and a positive integer always

# And feed in the data file
parameters['infile'] = GROUPS_INTEGRATIONS_DIR
input_dict = parameters.copy()

In [7]:
# Bookeeping for new/old parameters
inputs = list(parameters.keys())

# Perform the calculation 
results = perform_calculation(parameters)
for key in results:
    if key in inputs:
        if key == 'infile':
            # hackers
            print('The input of infile was REDACTED!')
        else:
            print('The input of {} was {}.'.format(key, results[key]))
    else:
        print('The result of {} was {}'.format(key, results[key]))

The input of mag was 8.637.
The input of band was k.
The input of mod was k5v.
The input of obs_time was 9.21592.
The input of n_group was 3.
The input of ins was niriss.
The input of filt was soss.
The input of subarray was substrip256.
The input of filt_ta was f480m.
The input of subarray_ta was im.
The input of sat_mode was well.
The input of sat_max was 44800.0.
The input of infile was REDACTED!
The result of n_col was 256
The result of n_row was 2048
The result of n_amp was 1
The result of n_reset was 1
The result of n_frame was 1
The result of n_skip was 0
The result of t_frame was 5.494
The result of t_int was 16.482
The result of t_ramp was 16.482
The result of n_int was 1510
The result of t_exp was 6.913
The result of t_duration was 9.218
The result of obs_eff was 0.75
The result of ta_t_frame was 0.05016
The result of min_ta_groups was 13
The result of max_ta_groups was 3
The result of t_duration_ta_min was 0.7022400000000001
The result of t_duration_ta_max was 0.20064
The re

In [8]:
# So let's make a nice printed summary 
print('The total time for science + TA observation scheme is {}-{} hours.'.format(
    results['t_duration']+results['t_duration_ta_max'], results['t_duration']+results['t_duration_ta_min']))
print('You need {} groups and {} integrations for the science observation.'.format(
    results['n_group'], results['n_int']))
print('You need between {} and {} groups for target acquisition.'.format(
    results['max_ta_groups'], results['min_ta_groups']))
print('We estimate your science observation will reach at most {} counts -- how close were we to your cutoff of {}?'.format(
    results['max_sat_prediction'], results['sat_max']))
print('With this observation scheme, {}% of the observation will be spent acquiring data.'.format(results['obs_eff']*100))

The total time for science + TA observation scheme is 9.41864-9.92024 hours.
You need 3 groups and 1510 integrations for the science observation.
You need between 3 and 13 groups for target acquisition.
We estimate your science observation will reach at most 40735.95 counts -- how close were we to your cutoff of 44800.0?
With this observation scheme, 75.0% of the observation will be spent acquiring data.


## Now we can put this data into APT to submit a proposal.
Please open your APT application now ...

# Generate Limb Darkening Coefficients
To generate limb darkening coefficients for our host star, we can use the `exoctk.limb_darkening` tool. We can create an `LDC` object using a stellar intensity grid from the Phoenix ACES or Kurucz ATLAS9 models (provided in the `EXOCTK_DATA` zip file).

In [4]:
from exoctk.limb_darkening.limb_darkening_fit import LDC
from exoctk.modelgrid import ModelGrid
from exoctk.utils import MODELGRID_DIR

#aces = ModelGrid(os.path.join(MODELGRID_DIR, 'ACES'), resolution=700)
#ld = LDC(aces)
atlas9 = ModelGrid(os.path.join(MODELGRID_DIR, 'ATLAS9'), resolution=700)
ld = LDC(atlas9)

141 models loaded from /Users/stevekb1/Documents/code/exoctk/exoctk_data/modelgrid/ATLAS9/


Next, we want to generate our limb darkening coefficients using a particular grism throughput using the `svo_filters` package. For this example, we'll use the GR700XD grism to simulate some NIRISS SOSS mode observations of our target, split into 15 equal wavelength bins.

In [5]:
from svo_filters import Filter
gr700xd = Filter('NIRISS.GR700XD.1', n_bins=15)
gr700xd.plot()

Bandpass trimmed to 0.54982 um - 2.799 um
15 bins of 149 pixels each.


Finally, we can calculate our limb darkening coefficients by passing the host star effective temperature, surface gravity, and metalicity to the `calculate` method. We will also provide the limb darkening profile name and the bandpass we wish to consider:

In [6]:
teff = wasp107b_params['Teff']
logg = wasp107b_params['stellar_gravity']
feh = wasp107b_params['Fe/H']
#print(teff, logg, feh)
ld.calculate(teff, logg, feh, 'quadratic', bandpass=gr700xd)
ld.results.pprint()

Loading flux into table...
Interpolating grid point [4430.0/4.5/0.02]...
Run time in seconds:  1.3212730884552002
  name   Teff  logg FeH   profile  ... color   c1    e1    c2    e2 
------- ------ ---- ---- --------- ... ----- ----- ----- ----- -----
0.63 um 4430.0  4.5 0.02 quadratic ...  blue 0.661 0.005 0.079 0.006
0.78 um 4430.0  4.5 0.02 quadratic ...  blue 0.492 0.009  0.14 0.011
0.93 um 4430.0  4.5 0.02 quadratic ...  blue 0.412  0.01 0.161 0.012
1.08 um 4430.0  4.5 0.02 quadratic ...  blue 0.361  0.01 0.177 0.012
1.23 um 4430.0  4.5 0.02 quadratic ...  blue 0.323  0.01 0.203 0.013
1.38 um 4430.0  4.5 0.02 quadratic ...  blue 0.271 0.013 0.259 0.016
1.52 um 4430.0  4.5 0.02 quadratic ...  blue 0.172  0.02 0.347 0.025
1.67 um 4430.0  4.5 0.02 quadratic ...  blue 0.096 0.026 0.371 0.033
1.82 um 4430.0  4.5 0.02 quadratic ...  blue 0.097 0.025 0.348 0.031
1.97 um 4430.0  4.5 0.02 quadratic ...  blue 0.103 0.023 0.324 0.028
2.12 um 4430.0  4.5 0.02 quadratic ...  blue 0.103 0.021 0

# Lightcurve Fitting
Let's say we extracted the 200 frames of time-series 1D spectra for our target and it looks like this:

In [24]:
# Generate the simulated light curve
from exoctk.lightcurve_fitting.simulations import simulate_lightcurve
npts = 1000
snr  = 4000.
wasp107b_unc = np.ones(npts)/snr
#simulate_lightcurve?
wasp107b_time, wasp107b_flux, foo,wasp107b_par = simulate_lightcurve('WASP-107b', snr=snr, npts=npts, nbins=1, plot=False)
print(wasp107b_flux.shape)

(1, 1000)


We can fit a transit model to the data with the `lightcurve_fitting.LightCurve` class.

In [25]:
from exoctk.lightcurve_fitting.lightcurve import LightCurve
wasp107b_lc = LightCurve(wasp107b_time, wasp107b_flux[0], unc=wasp107b_unc, name='WASP-107b')

Now we need to create a transit model to fit to the data. We'll define the orbital parameters and then feed it to the transit model.

In [26]:
# Set the intial parameters
from exoctk.lightcurve_fitting.parameters import Parameters
params = Parameters()
print(wasp107b_par)
params.rp = wasp107b_par['rp']+0.02, 'free', 0.0, 0.4
params.per = wasp107b_par['per'], 'fixed'
params.t0 = wasp107b_par['t0']-0.01, 'free', wasp107b_par['t0']-0.1, wasp107b_par['t0']+0.1
params.inc = wasp107b_par['inc'], 'free', 80., 90.
params.a = wasp107b_par['a'], 'free', 10., 25.
params.ecc = wasp107b_par['ecc'], 'fixed'
params.w = wasp107b_par['w'], 'fixed'
params.limb_dark = 'quadratic', 'independent'
params.transittype = 'primary', 'independent'
params.u1 = 0.1, 'free', 0., 1.
params.u2 = 0.1, 'free', 0., 1.

# Make the transit model
from exoctk.lightcurve_fitting.models import TransitModel
t_model = TransitModel(parameters=params, name='transit', fmt='r--')

# Plot it
t_model.plot(wasp107b_time, draw=True)

{'canonical_name': 'WASP-107 b', 'exoplanetID': 6435, 'catalog_name': 'nexsci', 'planet_name': 'WASP-107 b', 'disposition': 'confirmed planet', 'modified_date': '2019-12-28 18:16:56.338000', 'star_name': 'WASP-107', 'component': 'b', 'Rs': 0.66, 'Rs_unit': 'R_sun', 'Rs_upper': 0.02, 'Rs_lower': 0.02, 'Rs_ref': 'Anderson et al. 2017', 'Rs_url': None, 'Ms': 0.69, 'Ms_unit': 'M_sun', 'Ms_upper': 0.05, 'Ms_lower': 0.05, 'Ms_ref': 'Anderson et al. 2017', 'Ms_url': None, 'Fe/H': 0.02, 'Fe/H_upper': 0.1, 'Fe/H_lower': 0.1, 'Fe/H_ref': 'Anderson et al. 2017', 'Fe/H_url': None, 'stellar_gravity': 4.5, 'stellar_gravity_upper': 0.1, 'stellar_gravity_lower': 0.1, 'stellar_gravity_ref': 'Anderson et al. 2017', 'stellar_gravity_url': None, 'Teff': 4430.0, 'Teff_unit': 'K', 'Teff_upper': 120.0, 'Teff_lower': 120.0, 'Teff_ref': 'Anderson et al. 2017', 'Teff_url': None, 'Vmag': 11.471, 'Vmag_unit': 'mag', 'Vmag_upper': None, 'Vmag_lower': None, 'Vmag_ref': None, 'Vmag_url': None, 'Jmag': None, 'Jmag_un

KeyError: 'rp'

Next we can fit the model to the data to extract our orbital parameters.

In [12]:
# Create a new model instance from the best fit parameters
wasp107b_lc.fit(t_model, fitter='lmfit', method='powell')

# Plot it
wasp107b_lc.plot()

[[Model]]
    Model(eval)
[[Fit Statistics]]
    # fitting method   = Powell
    # function evals   = 73
    # data points      = 1000
    # variables        = 6
    chi-square         = 719041.869
    reduced chi-square = 723.382162
    Akaike info crit   = 6589.91959
    Bayesian info crit = 6619.36612
##  Warning: uncertainties could not be estimated:
    per:  at initial value
    t0:   at initial value
    t0:   at boundary
    ecc:  at initial value
    w:    at initial value
[[Variables]]
    rp:   0.33227827 (init = 0.166359)
    per:  5.72149 (fixed)
    t0:   57583.8905 (init = 57583.82)
    inc:  80.7469808 (init = 90)
    a:    21.0806866 (init = 17.91935)
    ecc:  0 (fixed)
    w:    90 (fixed)
    u1:   0.99798367 (init = 0.1)
    u2:   0.99822501 (init = 0.1)



# Atmospheric Retrievals 
TBD whether this will be part of this notebook, or on our little AWS token situations. 